In [ ]:
#importing really useful stuff
import rebound
import numpy as np
import matplotlib as plt
from math import log10, floor


# SECTION 1 - finding transit timing variations with various parameters

Section 1.0: finding and removing all unstable systems

Unstable systems will result in significantly increased transit timing variations, which could skew the averaged data. 

The critical a value (semi-major axis) for "venus" was determined to the value when the aphelion of "venus"'s orbit woukd cross with the perihelion of "earth"'s orbit for each value of 'e' (eccentricity). 

a_venus(1 + e_venus) = a_earth(1 - e_earth) --> a_venus = 1(1 - 0.05)/1 + e_venus

This ruled out a number of cases. However. there were still many unstable orbits on the boundary of that line (imagine a 2-dimensional graph with 'e' represented on the x axis and 'a' on the y axis', with the line represented at a=0.95/1+e). 

Due to this we ran trial cases (running the orbit systems for 100,000 years with the minimum exit distance set as 0.05 AU) for e = 0.05, e=0.1, e=0.15 and e=0.2. All cases for 'a' were run starting with a = 0.85 and running until there were 7-16 stable cases in a row. 'f' (anomaly) and 'omega' (location of particle) were randomized for each case between 0.0 and 2pi, which is why many stable orbits had to be seen in a row. the lowest percent of the critical a value which was a boundary for stable vs unstable cases was found to be ~0.94 (in other words, all cases lower than 0.94 x 'a crit' for each 'e' value were stable). the cutoff for running ttv variations was set to 0.93 x 'a crit'. all cases that remained with transit variation (in hours) greater than 9 hours were retested (running the orbit system for 100,000 years, minimum exit distance also 0.05 AU) and all were shown to be stable without exception.

In [ ]:
#defining how to find TTV

def runsimulation():
    
    #setup simulation
    sim = rebound.Simulation()
    sim.integrator = "ias15"
    sim.exit_min_distance = 0.05 #if the two particles ("earth" and "venus") reach within 0.05AU of each other, a rebound.encounter error is shown
    sim.add(m=1)
    sim.add(m=m_mass, a=a_distance, e=e_eccentricity, f=f_anomaly, omega=omega_locationofparticle)
    sim.add(m=3e-6, a=1,e=0.05,omega=0.25)
    sim.move_to_com()
     
    
    stable = True
    if a_distance*(1+e_eccentricity) < 1*(1-0.05): #a(1-e) for earth. if the aphelion of venus is less than the perihelion of earth, otherwise assume they're going to be unstable as the orbits cross
        print("running")
        nyears = 100000 #number of years the program runs for
        noutput = 10 #number of total steps, doesn't matter because the program checks every timestep anyways, this just sets an amount of time to integrate for
        times = np.linspace(0,nyears*2*np.pi,noutput)
        min_distance = 5
        distances = np.zeros(noutput)
        try:
            for i,time in enumerate(times):
                sim.integrate(time)
        except rebound.Encounter as error:
            print(a_distance, f_anomaly, omega_locationofparticle, "UNSTABLE") #important to print f and omega because they are randomized
            stable = False
    else: #if the orbits cross
        stable = False
        print(a_distance, "unstable. case omitted: orbits cross.")
        
    if stable == True:
        print(a_distance, f_anomaly, omega_locationofparticle, "STABLE")
    

In [ ]:
#change parameters
import numpy as np
import random
m_mass = 2.45e-6
e_eccentricity = 0.05 #0.10, 0.15, 0.20 
for i in range(85):
    a_distance = 0.85 - i*0.0015
    f_anomaly = random.uniform(0, 2*np.pi) #chooses a random value between 0 and 2pi
    omega_locationofparticle = random.uniform(0, 2*np.pi)
    runsimulation()

Section 1.1: running all systems and finding transit timing variations

with 'a' running from 0.7-0.85 AU, 'e' from 0.0-0.2, and 'f' and 'omega' from 0.0-2pi, each system was run for 19 years (4 years kepler, 11 years pause, 4 years mission). these 19 variations along with the 'a' and 'e' of each case were recorded in a file (ttvdata2.txt). 

In [ ]:
ttvdata = open("ttvdata.txt", "w+")  #currently the data is actually stored in ttvdata2.txt
ttvdata.close()

def ttv():
    #setup simulation
    sim = rebound.Simulation()
    sim.add(m=1) #this is the sun
    sim.add(m=m_mass, a=a_distance, e=e_eccentricity, f=f_anomaly, omega=omega_locationofparticle) #"venus", whatever parameters are currently running
    sim.add(m=3e-6, a=1,e=0.05,omega=0.25)
    sim.move_to_com()
    p = sim.particles
    
    addtottv = True
    
    #code below is not mine, from hannorein?

    #find individual time of intersection N times
    N=19
    transittimes = np.zeros(N)
    i = 0
    while i<N:
        y_old = p[2].y - p[0].y  
        t_old = sim.t
        sim.integrate(sim.t+0.5) # check for transits every 0.5 time units. Note that 0.5 is shorter than one orbit
        t_new = sim.t
        if y_old*(p[2].y-p[0].y)<0. and p[2].x-p[0].x>0.:   # sign changed (y_old*y<0), planet in front of star (x>0)
            while t_new-t_old>1e-7:   # bisect until prec of 1e-5 reached
                if y_old*(p[2].y-p[0].y)<0.:
                    t_new = sim.t
                else:
                    t_old = sim.t
                sim.integrate( (t_new+t_old)/2.)
            transittimes[i] = sim.t
            i += 1
            sim.integrate(sim.t+0.05)  

    
    #remove linear trend
    A = np.vstack([np.ones(N), range(N)]).T
    c, m = np.linalg.lstsq(A, transittimes)[0]  
    
    
    if a_distance > 0.93*(0.95/1+e_eccentricity):
        addtottv = False

    def round_sig(x, sig=2):
        return round(x, sig-int(floor(log10(abs(x))))-1)

    #if orbits are stable, adds the transit variations to a .txt file
    if addtottv == True:
        #store values in variable called temp. then convert to tempvconvert (each variation on indv. line, 4 sig. figs.)
        temp = (transittimes-m*np.array(range(N))-c)*(24.*365./2./np.pi);
        tempconvert = "\n" + str(a_distance) + "\n" + str(e_eccentricity)
        for i in range(len(temp)):
            tempconvert = tempconvert + "\n" + str(round_sig(temp[i], 4))
    
        #add tempconvert data to a .txt file
        ttvdata = open("ttvdata.txt","a+") 
        ttvdata.write(tempconvert)
        ttvdata.close()

In [ ]:
#MAIN (running parameters)
a_distance_min = 0.7
a_distance_max = 0.85
a_distance_total_steps = 100
a_distance_step = (a_distance_max - a_distance_min)/a_distance_total_steps

e_eccentricity_min = 0.0
e_eccentricity_max = 0.2
e_eccentricity_total_steps = 5 
e_eccentricity_step = (e_eccentricity_max - e_eccentricity_min)/e_eccentricity_total_steps

f_anomaly_min = 0.0
f_anomaly_max = 2*np.pi
f_anomaly_total_steps = 10
f_anomaly_step = (f_anomaly_max - f_anomaly_min)/f_anomaly_total_steps

omega_locationofparticle_min = 0.0
omega_locationofparticle_max = 2*np.pi
omega_locationofparticle_total_steps = 10
omega_locationofparticle_step = (omega_locationofparticle_max - omega_locationofparticle_min)/omega_locationofparticle_total_steps

m_mass = 2.45e-6
for i in range(a_distance_total_steps):
    a_distance = a_distance_min + i*a_distance_step 
    for j in range(e_eccentricity_total_steps):
        e_eccentricity = e_eccentricity_min + j*e_eccentricity_step
        for k in range(f_anomaly_total_steps): 
            f_anomaly = f_anomaly_min + k*f_anomaly_step
            for l in range(omega_locationofparticle_total_steps):
                omega_locationofparticle = omega_locationofparticle_min + l*omega_locationofparticle_step
                ttv()

# SECTION 2 - finding delta SNR between light transits with and without transit timing variations

section 2.0: Retrieving yearly transit timing variation data
The transit variation data from the file was read. Only the first 4 and last 4 variations of the 19 years was stored in an array (accounting for the missing 11 years). both 'a' and 'e' for each case were stored too for later use

In [ ]:
#the file containing the simulation and transit timing variations is opened and the information is added to a list
file1 = open('ttvdata.txt', 'r') 
mylist1 = file1.readlines() #adds all lines of txt file to list

#list 2 is created to strip each element of list1 of 'n' and any unneeded spaces
mylist2 = []
for i in range(len(mylist1)):
    mylist2.append(mylist1[i].strip())
mylist2.pop(0) 
#^^removes first element of list, which is an empty line (just happens to be the case in the particular text file)

#here another list is created to float each element of list2, which were strings
mylist3 = []
for i in range(len(mylist2)):
    mylist3.append(float(mylist2[i]))

#this create a 2-dimensional array containing the amount of cases and 21 elements each, inside which each of 
#the variations will be stored to use later
ttvarrayfake = np.zeros((len(mylist3)/21, 21))
 
#here, each of the elements in the list (the transit timing variations) are split up into cases (based on parameters)
#and assigned one of ten spots (corresponding to the year in the simulation they occured)
count = 0
for i in range(len(mylist3)/21): 
    for j in range(21): #used to be 19
        ttvarrayfake[i,j] = mylist3[count]
        count = count+1
        
ttvarray = np.zeros((len(mylist3)/21,10))
        
#removes the 11 years for which we do not have data (5-15 inclusive), first two are a and e
for i in range(len(mylist3)/21):
    for b in range(2):
        ttvarray[i,b] = ttvarrayfake[i,b]
    for k in range(4):
        ttvarray[i,2+k] = ttvarrayfake[i,2+k]
    for p in range(4):
        ttvarray[i,6+p]= ttvarrayfake[i,17+p]

a model of light levels (0=normal) was drawn for the "sun" over the course of 8 years for both a system without venus and a system with "venus". random noise (with blah blah blah of 5e-5?) was created (same noise was used for both cases). these 8 years were then averaged into one year, shallowing out the "box"

a box model was created to find the best fit. the box model ran through a _?_ grid of depths and distances to find the box model with the smallest chi square to the averaged data of the 8 years plotted over one year (shallowed out)

this best fitting box model was then used to fin the SNR for both venus and without venus for each case, using the formula ____ . a, e and delta snr for each case were then stored in an array and finally copied over to a txt file

In [ ]:
def findbox():
    #create an array that contains all the average values throughout a year
    averagesarray = np.zeros(int(halfhoursinyear))
    averagesarray1 = np.zeros(int(halfhoursinyear))
    for a in range(halfhoursinyear): 
        tempaverage = 0
        tempaverage1 = 0
        for b in range(numberofyears):
            tempaverage = tempaverage + myarray[b,1,a]
            tempaverage1 = tempaverage1 + myarray1[b,1,a]
        averagesarray[a] = tempaverage/numberofyears
        averagesarray1[a] = tempaverage1/numberofyears
        
    #create an initial value for setfit, for which tempfit will be lower
    setfit=10e10
    setfit1 =10e10
    
    #set boxarray variables to global so other segments can access the information. create a boxarray to represent the box
    global boxarray
    global boxarray1
    
    boxarray = np.zeros(int(halfhoursinyear))
    boxarray1 = np.zeros(int(halfhoursinyear))
    
    global tempdistance, tempdistance1
    global tempdepth, tempdepth1
    
    #try different variations of depth/distance (distance = transit duration) of the box
    depth_run = 40
    depth_min=0.4
    depth_max=1.2
    depth_step=(depth_max-depth_min)/depth_run
    distance_run = 40  
    distance_min=0.5
    distance_max=3
    distance_step=(distance_max-distance_min)/distance_run
    for i in range(depth_run):
        tempdepth = (depth_min + depth_step*i)*depthoftransit 
        for j in range(distance_run):
            tempdistance = (distance_min + distance_step*j)*transitduration
            
            trybox()
           
            mylist = np.zeros(halfhoursinyear)
            mylist1 = np.zeros(halfhoursinyear)
            
            mylist = testarray - averagesarray
            mylist1 = testarray - averagesarray1
            
            
            tempfit = np.sum(mylist**2)
            tempfit1 = np.sum(mylist1**2)
            
            if tempfit < setfit:
                setfit = tempfit
                setdepth = tempdepth
                setdistance = tempdistance
                boxarray = testarray
                
            if tempfit1 < setfit1:
                setfit1 = tempfit1
                setdepth1 = tempdepth
                setdistance1 = tempdistance
                boxarray1 = testarray

In [ ]:
def trybox():
    global testarray
    
    testarray = np.zeros(int(halfhoursinyear))
    
    #creates a beautiful box
    ta_midpoint = midpoint
    ta_tempsp = ta_midpoint - tempdistance/2
    ta_tempep = ta_midpoint + tempdistance/2
    
    ta_decimalsp = ta_tempsp - int(ta_tempsp)
    ta_decimalep = ta_tempep - int(ta_tempep)

    #using the cutoff between bins as ?.5. for example, if the transit was found to start at hour 25.4, the bin of
    #light for 25 would be set to 0.9 and for 26 would be set to full value.
    if ta_decimalsp > 0.50:
        ta_sp = int(ta_tempsp)+2 #start point of full value transits
        ta_percentsp = 1 - (ta_decimalsp - 0.50)
        ta_beforesp = int(ta_tempsp)+1 #the start point that doesn't have full value
        testarray[ta_beforesp] = 0 - tempdepth*ta_percentsp
    elif ta_decimalsp < 0.50:
        ta_sp = int(ta_tempsp)+1
        ta_percentsp = 1 - (0.50-ta_decimalsp)
        ta_beforesp = int(ta_tempsp)
        testarray[ta_beforesp] = 0 - tempdepth*ta_percentsp
     
    if ta_decimalep < 0.50:
        ta_ep = int(ta_tempep)-1
        ta_percentep = 1 - (0.50-ta_decimalep)
        ta_afterep = int(ta_tempep)
        testarray[ta_afterep] = 0 - tempdepth*ta_percentep
    if ta_decimalep > 0.50:
        ta_ep = int(ta_tempep)
        ta_percentep = 1 - (ta_decimalep - 0.50)
        ta_afterep = int(ta_tempep)+1
        testarray[ta_afterep] = 0 - tempdepth*ta_percentep

    #endpoint-startpoint + 1(to be inclusive)
    ta_actual_length_of_transit = ta_ep-ta_sp+1 
        
    #the non-full transit depths have been set, here the full transit depths are set
    for z in range(ta_actual_length_of_transit):
        testarray[ta_sp+z] = 0 - tempdepth
        

In [ ]:
#now run everything
casenumbers = len(ttvarray)

#will contain a, e and delta SNR
valuesarray = np.zeros((casenumbers,3))

for cn in range(casenumbers):
    numberofyears = 8
    lengthoforbit = 1
    transitduration = 12.9791194828*2 #in half hours
    midpoint = 50 #halfhours
    halfhoursinyear = 17520*lengthoforbit
    depthoftransit = 0.00008389575
    sigma = 1*(5e-5)

    myarray = np.zeros((numberofyears,2,int(halfhoursinyear)))
    myarray1 = np.zeros((numberofyears,2,int(halfhoursinyear)))
    #seperated by number of years (0=1-9=10), time=0, light=1
    
    withoutvenusarray = np.zeros(int(halfhoursinyear)) #creates an array with one space for each bin in the number of years
    withvenusarray = np.zeros(int(halfhoursinyear)) #creates another one, this one for with ttv

    randomnoisearray = np.zeros((numberofyears, int(halfhoursinyear)))
    
    for i in range(numberofyears):
        
        #Section 2.2: creating the light levels over the course of 8 years
       
        #RUN WITHOUT VENUS VARIATIONS
        #setting all time to increase by 1/17520th of a year (half hour) increments within the period of each year
        for j in range(len(myarray1[i,0])):
            myarray1[i,0,j]= 0 + j*0.5    #0.00005707762
   
        #setting all light levels = 1 (mean) 
        for k in range(len(myarray1[i,1])):
                myarray1[i,1,k]=0

        #subtracting the light decrease resulting from transit
        ttvsp1 = midpoint - transitduration/2 #transit start point
        ttvep1 = midpoint + transitduration/2 #transit end point
        
       
        decimalsp1 = (ttvsp1 - int(ttvsp1))
        decimalep1 = (ttvep1 - int(ttvep1))
        
        #basically determines which side of the 0.5 boundary the start point and end point 
        #are and therefore which bin/half hour is the start of the full transit and which might have some 
        #fractional transit
        if decimalsp1 > 0.50:
            sp1 = int(ttvsp1)+2 #start point of full value transits
            percentsp1 = 1 - (decimalsp1 - 0.50)
            beforesp1 = int(ttvsp1)+1 #the start point that doesn't have full value
            myarray1[i, 1, beforesp1] = myarray1[i,1,beforesp1] - depthoftransit*percentsp1
        elif decimalsp1 < 0.50:
            sp1 = int(ttvsp1)+1
            percentsp1 = 1 - (0.50-decimalsp1)
            beforesp1 = int(ttvsp1)
            myarray1[i, 1, beforesp1] = myarray1[i,1,beforesp1] - depthoftransit*percentsp1
        
        if decimalep1 < 0.50:
            ep1 = int(ttvep1)-1
            percentep1 = 1 - (0.50-decimalep1)
            afterep1 = int(ttvep1)
            myarray1[i, 1, afterep1] = myarray1[i,1,afterep1] - depthoftransit*percentep1
        if decimalep1 > 0.50:
            ep1 = int(ttvep1)
            percentep1 = 1 - (decimalep1 - 0.50)
            afterep1 = int(ttvep1)+1
            myarray1[i, 1, afterep1] = myarray1[i,1,afterep1] - depthoftransit*percentep1
        
        actual_length_of_transit1 = ep1-sp1+1
        
        for z in range(actual_length_of_transit1):
            myarray1[i,1,sp1+z] = myarray1[i,1,sp1+z] - depthoftransit

        #adding noise
        for m in range(len(myarray1[i,1])):
            randomnoisearray[i,m] = np.random.normal(0,sigma) #mean is 1, range of distribution is 0.00005 either direction (e-5)
            myarray1[i,1,m] = myarray1[i,1,m] + randomnoisearray[i,m]

        
        #RUN WITH VENUS VARIATIONS
        #setting all time to increase by 1/17520th of a year (half hour) increments within the period of each year
        for j in range(len(myarray[i,0])):
            myarray[i,0,j]= 0 + j*0.5    #0.00005707762
       
        #setting all light levels = 0 (mean) 
        for k in range(len(myarray[i,1])):
                myarray[i,1,k]=0

        #subtracting the light decrease resulting from transit
        ttvmidpoint = midpoint + ttvarray[cn, 2+i] #changed it because the first two values are a and e

        ttvsp = ttvmidpoint - transitduration/2 #transit start point
        ttvep = ttvmidpoint + transitduration/2 #transit end point
        
       
        decimalsp = (ttvsp - int(ttvsp))
        decimalep = (ttvep - int(ttvep))
        
        if decimalsp > 0.50:
            sp = int(ttvsp)+2 #start point of full value transits
            percentsp = 1 - (decimalsp - 0.50)
            beforesp = int(ttvsp)+1 #the start point that doesn't have full value
            myarray[i, 1, beforesp] = myarray[i,1,beforesp] - depthoftransit*percentsp
        elif decimalsp < 0.50:
            sp = int(ttvsp)+1
            percentsp = 1 - (0.50-decimalsp)
            beforesp = int(ttvsp)
            myarray[i, 1, beforesp] = myarray[i,1,beforesp] - depthoftransit*percentsp
        
        
        if decimalep < 0.50:
            ep = int(ttvep)-1
            percentep = 1 - (0.50-decimalep)
            afterep = int(ttvep)
            myarray[i, 1, afterep] = myarray[i,1,afterep] - depthoftransit*percentep
        if decimalep > 0.50:
            ep = int(ttvep)
            percentep = 1 - (decimalep - 0.50)
            afterep = int(ttvep)+1
            myarray[i, 1, afterep] = myarray[i,1,afterep] - depthoftransit*percentep
        
        actual_length_of_transit = ep-sp+1
        
        for z in range(int(actual_length_of_transit)):
            myarray[i,1,sp+z] = myarray[i,1,sp+z] - depthoftransit
        
    
        midpoint = midpoint + halfhoursinyear - int(halfhoursinyear)

        #adding noise
        for m in range(len(myarray[i,1])):
            myarray[i,1,m] = myarray[i,1,m] + randomnoisearray[i,m]
          
    
    #SECTION 2.3: finding the best fitting box model. finds the depth and distance(transit duration) of a box that 
    #has the lowest chi square of an array 
    findbox()

    #SECTION 2.4: finding signal to noise ratio
    for m in range(halfhoursinyear):
        withvenusarray[m] = ((0 - boxarray[m])/sigma)**2
        withoutvenusarray[m] = ((0 - boxarray1[m])/sigma)**2
    
    SNR1 = np.sqrt(np.sum(withoutvenusarray))
    #print("signal to noise ratio without venus", SNR1)
    SNR2 = np.sqrt(np.sum(withvenusarray))
    #print("   signal to noise ratio with venus", SNR2)

    differenceinsnr = SNR1 - SNR2 #without venus - with venus

    #adds a,e and delta SNR to values array
    valuesarray[cn,0] = ttvarray[cn,0]
    valuesarray[cn,1] = ttvarray[cn,1]
    valuesarray[cn, 2] = differenceinsnr
    

Section 2.5: a, e and delta SNR for each case (recorded in values array) are copied over to valuesarray.txt file

In [ ]:
myfile = open("valuesarray.txt", "w+") #information actually in valuesarray2.txt
myfile.close()

myfile = open("valuesarray.txt","a+") 
for i in range(len(valuesarray)):
    tempfile = "\n" + str(valuesarray[i,0]) + "\n" + str(valuesarray[i,1]) + "\n" + str(valuesarray[i,2])
    myfile.write(tempfile)

myfile.close()

# SECTION 3 - plotting the data in 3d matplotlib scatterplot

both a,e,delta SNR (averaged for each 100 cases with same a and e) and a,e,ttv (averaged for each 100 cases with same a and e) were plotted seperately in two graphs. then the highest ttv was run by itself and averaged over 500 times. the result was 1.7, while the corresponding value of delta SNR was around 1.5. this was done to confirm the abscence of any gross coding error

In [ ]:
file1 = open('valuesarray2.txt', 'r') 
mylist1 = file1.readlines() #adds all lines of txt file to list

#list 2 is created to strip each element of list1 of 'n' and any unneeded spaces
mylist2 = []
for i in range(len(mylist1)):
    mylist2.append(mylist1[i].strip())
mylist2.pop(0) 
#^^removes first element of list, which is an empty line (just happens to be the case in the particular
#text file)

#here another list is created to float each element of list2, which were strings
mylist3 = []
for i in range(len(mylist2)):
    mylist3.append(float(mylist2[i]))

valuesarray = np.zeros((len(mylist3)/3,3))

count = 0
for i in range(len(mylist3)/3):
    valuesarray[i,0] = mylist3[count]
    valuesarray[i,1] = mylist3[count+1]
    valuesarray[i,2] = mylist3[count+2]
    count = count + 3

print(valuesarray[1])

In [ ]:
#creates a plot with a, e and SNR average change for each 100 cases with same a and e

import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

#this commented out section is to produce a figure with not the averaged values (for each same a and e) but for every single case
# xlist = []
# ylist = []
# zlist = []

# for i in range(len(valuesarray)):
#     xlist.append(valuesarray[i,0])
#     ylist.append(valuesarray[i,1])
#     zlist.append(valuesarray[i,2])

    
# ax.scatter(xlist, ylist, zlist, c=np.abs(zlist),cmap="YlGn")

# ax.set_xlabel('distance')
# ax.set_ylabel('eccentricity')
# ax.set_zlabel('delta SNR')

# plt.show()

averagearray = np.zeros((len(valuesarray)/100,3))
for k in range(len(averagearray)):
    tempzaverage = 0
    for i in range(100):
        tempzaverage = tempzaverage + valuesarray[k*100+i,2]
    tempzaverage = tempzaverage/100
    averagearray[k,0] = np.sqrt((valuesarray[k*100,0])**3)
    averagearray[k,1] = valuesarray[k*100,1]
    averagearray[k,2] = tempzaverage

xlist = []
ylist = []
zlist = []

for i in range(len(averagearray)):
    xlist.append(averagearray[i,0])
    ylist.append(averagearray[i,1])
    zlist.append(averagearray[i,2])
    
p = ax.scatter(xlist, ylist, zlist, c=np.abs(zlist),cmap="brg")

fig.colorbar(p)

plt.show()

In [ ]:
#creates a plot with a, e and average abs(ttv change) for each 100 cases with same a and e


import numpy as np

file1 = open('ttvdata2.txt', 'r') 
mylist1 = file1.readlines() 

#list 2 is created to strip each element of list1 of 'n' and any unneeded spaces
mylist2 = []
for i in range(len(mylist1)):
    mylist2.append(mylist1[i].strip())
mylist2.pop(0) 
#^^removes first element of list, which is an empty line (just happens to be the case in the particular
#text file)

#here another list is created to float each element of list2, which were strings
mylist3 = []
for i in range(len(mylist2)):
    mylist3.append(float(mylist2[i]))

    
ttvarray1 = np.zeros((len(mylist3)/21, 21))
 
count = 0
for i in range(len(mylist3)/21): 
    for j in range(21): 
        ttvarray1[i,j] = mylist3[count]
        count = count+1
        
ttvarray2 = np.zeros((len(mylist3)/21,10))
        
#removes the 11 years for which we do not have data (5-15 inclusive), first two are a and e
for i in range(len(mylist3)/21):
    for b in range(2):
        ttvarray2[i,b] = ttvarray1[i,b]
    for k in range(4):
        ttvarray2[i,2+k] = ttvarray1[i,2+k]
    for p in range(4):
        ttvarray2[i,6+p]= ttvarray1[i,17+p]

        
#finding the average of each case
ttvarray3 = np.zeros((len(mylist3)/21,3))

for i in range(len(ttvarray2)):
    ttvarray3[i,0] = ttvarray2[i,0]
    ttvarray3[i,1] = ttvarray2[i,1]
    ttvaverage = 0
    for j in range(8):
        ttvaverage = ttvaverage + abs(ttvarray2[i,j+2])
    ttvaverage = ttvaverage/8
    ttvarray3[i,2] = ttvaverage


#finding the average of the 100 cases with same a and e in ttvarray 3
ttvarray = np.zeros((len(ttvarray3)/100, 3))

for i in range(len(ttvarray3)/100):
    ttvarray[i,0] = ttvarray3[i*100,0]
    ttvarray[i,1] = ttvarray3[i*100,1]
    ttvaverage1 = 0
    for j in range(100):
        ttvaverage1 = ttvaverage1 + ttvarray3[i*100+j,2]
    ttvaverage1 = ttvaverage1/100
    ttvarray[i,2] = ttvaverage1


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

xlist = []
ylist = []
zlist = []

for i in range(len(ttvarray)):
    xlist.append(ttvarray[i,0])
    ylist.append(ttvarray[i,1])
    zlist.append(ttvarray[i,2])
    

    
p = ax.scatter(xlist, ylist, zlist, c=np.abs(zlist),cmap="brg")

fig.colorbar(p)

plt.show()

temporary (averaged over 100) of 8.5 hours ttv was 1.70698076211 SNR difference 